<a href="https://colab.research.google.com/github/asm24legend/IITGuwahatiSummerProject25/blob/main/Summer_Project'25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ipython sql


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 38.8 MB/s eta 0:00:00
  Created wheel for sql: filename=sql-2022.4.0-py3-none-any.whl size=4306 sha256=0d7cb30530bf961e90923bf56421419ecc775da85b27ef6455750a003900676a
  Stored in directory: /root/.cache/pip/wheels/a6/f1/62/be6faba20c8384c5766c0332c93841b610ab2602d6ae312bbf
Successfully built sql


In [2]:
import pandas as pd
import sqlite3

In [3]:
df = pd.read_csv("/content/inventory_forecasting.csv")

In [4]:
conn = sqlite3.connect('inventory_forecasting.db')
df.to_sql('inventory_forecasting', conn, if_exists='replace', index=False)

109500

In [5]:
#Load the SQL extension so that it can be used anywhere in the code
%load_ext sql

In [6]:
#Connect to created SQL database
%sql sqlite:///inventory_forecasting.db

In [7]:
query = "SELECT * FROM inventory_forecasting LIMIT 20;"
df = pd.read_sql_query(query, conn)
df


,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,Discount,Weather Condition,Holiday/Promotion,Competitor Pricing,Seasonality
0,2022-01-01,S001,P0096,Toys,West,158,134,142,152.36,40.88,5,Sunny,1,42.39,Winter
1,2022-01-01,S001,P0016,Clothing,East,189,127,125,150.47,90.78,0,Rainy,0,84.16,Winter
2,2022-01-01,S001,P0031,Electronics,West,75,48,39,68.62,13.99,20,Rainy,0,12.67,Winter
3,2022-01-01,S001,P0159,Electronics,West,161,74,71,75.46,84.92,10,Snowy,0,83.03,Winter
4,2022-01-01,S001,P0129,Furniture,South,135,66,69,67.18,19.66,0,Cloudy,0,20.55,Winter
5,2022-01-01,S001,P0116,Furniture,East,53,63,52,71.15,92.26,10,Cloudy,0,88.54,Winter
6,2022-01-01,S001,P0070,Electronics,North,99,59,51,69.07,78.39,0,Cloudy,1,79.78,Winter
7,2022-01-01,S001,P0171,Electronics,East,133,59,59,67.23,57.84,5,Cloudy,0,59.39,Winter
8,2022-01-01,S001,P0175,Electronics,East,163,88,86,112.45,18.62,0,Cloudy,0,19.25,Winter
9,2022-01-01,S001,P0046,Clothing,South,139,117,110,128.42,64.72,5,Rainy,0,67.55,Winter


In [8]:
#print all column names
df.columns

Index(['Date', 'Store ID', 'Product ID', 'Category', 'Region',
       'Inventory Level', 'Units Sold', 'Units Ordered', 'Demand Forecast',
       'Price', 'Discount', 'Weather Condition', 'Holiday/Promotion',
       'Competitor Pricing', 'Seasonality'],
      dtype='object')

In [9]:
cursor = conn.cursor()

In [10]:
# Close any open %sql connections to the database
%sql --close sqlite:///inventory_forecasting.db

# ---------------- DROP existing normalized tables if needed ----------------
cursor.execute("DROP TABLE IF EXISTS InventoryFacts;")
cursor.execute("DROP TABLE IF EXISTS MarketConditions;")
cursor.execute("DROP TABLE IF EXISTS Dates;")
cursor.execute("DROP TABLE IF EXISTS Stores;")
cursor.execute("DROP TABLE IF EXISTS Products;")

In [11]:
# Products Table
cursor.execute('''
CREATE TABLE Products (
    product_id TEXT PRIMARY KEY,
    category TEXT
);
''')

In [12]:
cursor.execute('''
INSERT OR IGNORE INTO Products (product_id, category)
SELECT DISTINCT "Product ID", "Category"
FROM inventory_forecasting;
''')
conn.commit()

In [13]:
# Stores Table with composite PK (store_id, region)
cursor.execute('''
CREATE TABLE Stores (
    store_id TEXT,
    region TEXT,
    PRIMARY KEY (store_id, region)
);
''')

In [14]:
cursor.execute('''
INSERT OR IGNORE INTO Stores (store_id, region)
SELECT DISTINCT "Store ID", "Region"
FROM inventory_forecasting;
''')

In [15]:
# Dates Table
cursor.execute('''
CREATE TABLE Dates (
    date DATE PRIMARY KEY,
    seasonality TEXT,
    is_holiday_or_promotion BOOLEAN
);
''')

In [16]:
cursor.execute('''
INSERT OR IGNORE INTO Dates (date, seasonality, is_holiday_or_promotion)
SELECT DISTINCT Date, Seasonality, "Holiday/Promotion"
FROM inventory_forecasting;
''')


In [17]:
# MarketConditions Table with composite FK (store_id, region)
cursor.execute('''
CREATE TABLE MarketConditions (
    store_id TEXT,
    region TEXT,
    date DATE,
    weather_condition TEXT,
    competitor_pricing REAL,
    created_at TIMESTAMP,
    PRIMARY KEY (store_id, region, date),
    FOREIGN KEY (store_id, region) REFERENCES Stores(store_id, region),
    FOREIGN KEY (date) REFERENCES Dates(date)
);
''')

In [18]:
cursor.execute('''
INSERT OR IGNORE INTO MarketConditions (
    store_id, region, date, weather_condition, competitor_pricing, created_at
)
SELECT DISTINCT
    "Store ID", "Region", Date, "Weather Condition", "Competitor Pricing", CURRENT_TIMESTAMP
FROM inventory_forecasting;
''')


In [19]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS InventoryFacts (
    date DATE,
    store_id TEXT,
    region TEXT,
    product_id TEXT,
    inventory_level INTEGER,
    units_sold INTEGER,
    units_ordered INTEGER,
    demand_forecast INTEGER,
    price REAL,
    discount REAL,
    created_at TIMESTAMP,
    PRIMARY KEY (date, store_id, region, product_id),
    FOREIGN KEY (product_id) REFERENCES Products(product_id),
    FOREIGN KEY (store_id, region) REFERENCES Stores(store_id, region),
    FOREIGN KEY (date) REFERENCES Dates(date)
);
''')
conn.commit()

In [20]:
cursor.execute('''
INSERT OR IGNORE INTO InventoryFacts (
    date, store_id, region, product_id, inventory_level,
    units_sold, units_ordered, demand_forecast, price, discount, created_at
)
SELECT DISTINCT
    Date, "Store ID", "Region", "Product ID", "Inventory Level",
    "Units Sold", "Units Ordered", "Demand Forecast", Price, Discount, CURRENT_TIMESTAMP
FROM inventory_forecasting;
''')
conn.commit()

In [21]:
#View tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
for table in tables:
    print(table[0])

inventory_forecasting
Products
Stores
Dates
MarketConditions
InventoryFacts


In [22]:
#Print the first 5 rows of Stores table
cursor.execute("SELECT * FROM Products LIMIT 5;")
rows = cursor.fetchall()
for row in rows:
    print(row)

('P0096', 'Toys')
('P0016', 'Clothing')
('P0031', 'Electronics')
('P0159', 'Electronics')
('P0129', 'Furniture')


In [23]:
#Print latest date from the Dates table
cursor.execute("SELECT MAX(date) FROM Dates;")
latest_date = cursor.fetchone()[0]
print("Latest date in Dates table:", latest_date)

Latest date in Dates table: 2023-12-31


In [24]:
#Print the different categories from the products table
cursor.execute("SELECT DISTINCT category FROM Products;")
categories = cursor.fetchall()
for category in categories:
    print(category[0])

Toys
Clothing
Electronics
Furniture
Groceries


In [25]:
#Compare the sales value = units_sold*(price-(discount/100)*price) by category
cursor.execute('''
SELECT
    p.category,
    SUM(
        i.units_sold * (
            i.price - (i.discount / 100) * i.price
        )
    ) AS sales_value
FROM
    InventoryFacts i
JOIN
    Products p ON i.product_id = p.product_id
    GROUP BY
        p.category
    ORDER BY
        sales_value DESC;
''')
sales_by_category = cursor.fetchall()
for category, sales_value in sales_by_category:
    print(f"Category: {category}, Sales Value: {sales_value}")

Category: Clothing, Sales Value: 215261243.08749995
Category: Electronics, Sales Value: 131131428.07349995
Category: Furniture, Sales Value: 103982170.46900003
Category: Toys, Sales Value: 51738873.288999945
Category: Groceries, Sales Value: 37328568.86049998


In [26]:
#Print the different store IDs from Stores
cursor.execute("SELECT DISTINCT store_id FROM Stores;")
store_ids = cursor.fetchall()
for store_id in store_ids:
    print(store_id[0])

S001
S002
S003
S004
S005


In [27]:
#Print highest priced items
cursor.execute('''
    SELECT
        p.product_id,
        p.category,
        MAX(i.price) AS highest_price
    FROM
        InventoryFacts i
    JOIN
        Products p ON i.product_id = p.product_id
    GROUP BY
        p.product_id, p.category
    ORDER BY
        highest_price DESC;
''')
highest_priced_items = cursor.fetchall()
for product_id, category, highest_price in highest_priced_items:
    print(f"Product ID: {product_id}, Category: {category}, Highest Price: {highest_price}")


Product ID: P0031, Category: Electronics, Highest Price: 100.0
Product ID: P0083, Category: Toys, Highest Price: 100.0
Product ID: P0116, Category: Furniture, Highest Price: 100.0
Product ID: P0133, Category: Clothing, Highest Price: 100.0
Product ID: P0149, Category: Furniture, Highest Price: 100.0
Product ID: P0178, Category: Clothing, Highest Price: 100.0
Product ID: P0046, Category: Clothing, Highest Price: 99.99
Product ID: P0061, Category: Clothing, Highest Price: 99.99
Product ID: P0066, Category: Clothing, Highest Price: 99.99
Product ID: P0096, Category: Toys, Highest Price: 99.99
Product ID: P0125, Category: Clothing, Highest Price: 99.99
Product ID: P0129, Category: Furniture, Highest Price: 99.99
Product ID: P0159, Category: Electronics, Highest Price: 99.99
Product ID: P0171, Category: Electronics, Highest Price: 99.99
Product ID: P0016, Category: Clothing, Highest Price: 99.98
Product ID: P0017, Category: Toys, Highest Price: 99.98
Product ID: P0079, Category: Furniture, 

In [28]:
#Calculate time period in days
cursor.execute('''
SELECT
    MAX(date) - MIN(date) AS time_period_in_days
FROM
    Dates;
''')
time_period_in_days = cursor.fetchone()[0]


In [29]:
#Calculate sales velocity=units sold/Time period(in days) by category
cursor.execute('''
SELECT
    p.category,
    CAST(SUM(i.units_sold) AS REAL) / ? AS sales_velocity
FROM
    InventoryFacts i
JOIN
    Products p ON i.product_id = p.product_id
GROUP BY
    p.category
ORDER BY
    sales_velocity DESC;
''', (time_period_in_days,))
sales_velocity_by_category = cursor.fetchall()
for category, sales_velocity in sales_velocity_by_category:
    print(f"Category: {category}, Sales Velocity: {sales_velocity}")

Category: Clothing, Sales Velocity: 4230880.0
Category: Electronics, Sales Velocity: 2577043.0
Category: Furniture, Sales Velocity: 2035351.0
Category: Toys, Sales Velocity: 1018493.0
Category: Groceries, Sales Velocity: 729550.0


In [30]:
#Print rows where inventory level<demand forecast by category
cursor.execute('''
SELECT
    p.category,
    i.inventory_level,
    i.demand_forecast
FROM
    InventoryFacts i
JOIN
    Products p ON i.product_id = p.product_id
    WHERE i.inventory_level < i.demand_forecast
    ORDER BY
        p.category;
''')
rows = cursor.fetchall()
for row in rows:
    print(row)

#Count the number of items per category after above query
cursor.execute('''
SELECT
    p.category,
    COUNT(*) AS item_count
FROM
    InventoryFacts i
    JOIN Products p ON i.product_id = p.product_id
    WHERE i.inventory_level < i.demand_forecast
    GROUP BY
        p.category;
''')
item_count_by_category = cursor.fetchall()
for category, item_count in item_count_by_category:
    print(f"Category: {category}, Item Count: {item_count}")

('Clothing', 210, 228.88)
('Clothing', 129, 131.15)
('Clothing', 166, 195.1)
('Clothing', 73, 77.65)
('Clothing', 189, 212.11)
('Clothing', 186, 194.53)
('Clothing', 109, 159.1)
('Clothing', 115, 139.64)
('Clothing', 84, 93.57)
('Clothing', 159, 172.28)
('Clothing', 194, 265.34)
('Clothing', 136, 174.09)
('Clothing', 110, 127.75)
('Clothing', 88, 89.91)
('Clothing', 155, 186.59)
('Clothing', 155, 173.04)
('Clothing', 131, 158.06)
('Clothing', 139, 171.99)
('Clothing', 112, 139.66)
('Clothing', 165, 180.26)
('Clothing', 101, 124.8)
('Clothing', 137, 161.49)
('Clothing', 176, 186)
('Clothing', 134, 142.86)
('Clothing', 161, 197.59)
('Clothing', 126, 156.99)
('Clothing', 129, 137.89)
('Clothing', 130, 178.62)
('Clothing', 191, 207.14)
('Clothing', 93, 133.65)
('Clothing', 81, 113.3)
('Clothing', 214, 271.94)
('Clothing', 91, 97.71)
('Clothing', 101, 142.28)
('Clothing', 133, 197.21)
('Clothing', 170, 174.68)
('Clothing', 92, 109.35)
('Clothing', 68, 70.92)
('Clothing', 125, 156.49)
('Clot

In [31]:
#Count the total number of items per category
cursor.execute('''
SELECT
    p.category,
    COUNT(*) AS item_count
FROM
    InventoryFacts i
    JOIN Products p ON i.product_id = p.product_id
    GROUP BY
        p.category;
''')
item_count_by_category = cursor.fetchall()
for category, item_count in item_count_by_category:
    print(f"Category: {category}, Item Count: {item_count}")


Category: Clothing, Item Count: 40150
Category: Electronics, Item Count: 29200
Category: Furniture, Item Count: 21900
Category: Groceries, Item Count: 7300
Category: Toys, Item Count: 10950


In [32]:
#Divide both to find stockout rate (for example for clothing Stockout rate=6383/40150 as obtained above)
query = '''
SELECT
    p.category,
    SUM(CASE WHEN i.inventory_level < i.demand_forecast THEN 1 ELSE 0 END) AS stockout_days,
    COUNT(*) AS total_days,
    ROUND(
        SUM(CASE WHEN i.inventory_level < i.demand_forecast THEN 1 ELSE 0 END) * 100.0 / COUNT(*),
        2
    ) AS stockout_rate_pct
FROM
    InventoryFacts i
JOIN
    Products p ON i.product_id = p.product_id
GROUP BY
    p.category
ORDER BY
    stockout_rate_pct DESC;
'''

df_category_stockouts = pd.read_sql_query(query, conn)
df_category_stockouts.head()



,category,stockout_days,total_days,stockout_rate_pct
0,Clothing,6383,40150,15.90
1,Groceries,1118,7300,15.32
2,Toys,1618,10950,14.78
3,Furniture,3178,21900,14.51
4,Electronics,3975,29200,13.61


In [33]:
#Stockouts by Store ID to find supplier reliability
query = '''
SELECT
    store_id,
    SUM(CASE WHEN inventory_level < demand_forecast THEN 1 ELSE 0 END) AS stockout_days,
    COUNT(*) AS total_days,
    ROUND(
        SUM(CASE WHEN inventory_level < demand_forecast THEN 1 ELSE 0 END) * 100.0 / COUNT(*),
        2
    ) AS stockout_rate_pct
FROM
    InventoryFacts
GROUP BY
    store_id
ORDER BY
    stockout_rate_pct DESC;
'''

df_store_stockouts = pd.read_sql_query(query, conn)
df_store_stockouts.head()


,store_id,stockout_days,total_days,stockout_rate_pct
0,S005,3312,21900,15.12
1,S003,3282,21900,14.99
2,S004,3258,21900,14.88
3,S002,3237,21900,14.78
4,S001,3183,21900,14.53


In [34]:
#Calculate stockout rate by region
query = '''
SELECT
    region,
    SUM(CASE WHEN inventory_level < demand_forecast THEN 1 ELSE 0 END) AS stockout_days,
    COUNT(*) AS total_days,
    ROUND(
        SUM(CASE WHEN inventory_level < demand_forecast THEN 1 ELSE 0 END) * 100.0 / COUNT(*),
        2
    ) AS stockout_rate_pct
FROM
    InventoryFacts
GROUP BY
    region
ORDER BY
    stockout_rate_pct DESC;
'''

df_region_stockouts = pd.read_sql_query(query, conn)
df_region_stockouts.head()

,region,stockout_days,total_days,stockout_rate_pct
0,North,4120,27293,15.10
1,South,4060,27218,14.92
2,West,4057,27430,14.79
3,East,4035,27559,14.64


In [35]:
#Calculate stockout rate by weather
query = '''
SELECT
    mc.weather_condition,
    SUM(CASE WHEN i.inventory_level < i.demand_forecast THEN 1 ELSE 0 END) AS stockout_days,
    COUNT(*) AS total_days,
    ROUND(
        SUM(CASE WHEN i.inventory_level < i.demand_forecast THEN 1 ELSE 0 END) * 100.0 / COUNT(*),
        2
    ) AS stockout_rate_pct
FROM
    InventoryFacts i
JOIN
    MarketConditions mc ON i.date = mc.date AND i.store_id = mc.store_id AND i.region = mc.region
GROUP BY
    mc.weather_condition
ORDER BY
    stockout_rate_pct DESC;
    '''

df_weather_stockouts = pd.read_sql_query(query, conn)
df_weather_stockouts.head()

,weather_condition,stockout_days,total_days,stockout_rate_pct
0,Cloudy,4015,26944,14.90
1,Snowy,4086,27466,14.88
2,Sunny,3988,26850,14.85
3,Rainy,4183,28240,14.81


In [36]:
#Calculate stockout rate by season
query = '''
SELECT
    d.seasonality,
    SUM(CASE WHEN i.inventory_level < i.demand_forecast THEN 1 ELSE 0 END) AS stockout_days,
    COUNT(*) AS total_days,
    ROUND(
        SUM(CASE WHEN i.inventory_level < i.demand_forecast THEN
        1 ELSE 0 END) * 100.0 / COUNT(*),
        2
    ) AS stockout
    FROM
        InventoryFacts i
    JOIN
        Dates d ON i.date = d.date
    GROUP BY
        d.seasonality
    ORDER BY
        stockout DESC;
'''

df_season_stockouts = pd.read_sql_query(query, conn)
df_season_stockouts.head()

,seasonality,stockout_days,total_days,stockout
0,Winter,5789,36000,16.08
1,Summer,4010,27600,14.53
2,Spring,2619,18300,14.31
3,Autumn,3854,27600,13.96


In [37]:
#Calculate the average demand by category according to season
query = '''
SELECT
    p.category,
    d.seasonality,
    AVG(i.demand_forecast) AS avg_demand
FROM
    InventoryFacts i
JOIN
    Products p ON i.product_id = p.product_id
    JOIN Dates d ON i.date = d.date
GROUP BY
    p.category, d.seasonality
ORDER BY
    p.category, d.seasonality
LIMIT 20;

'''
#Print all rows
df_season_demand = pd.read_sql_query(query, conn)
df_season_demand.head(25)



,category,seasonality,avg_demand
0,Clothing,Autumn,103.839405
1,Clothing,Spring,104.455973
2,Clothing,Summer,104.088378
3,Clothing,Winter,145.284611
4,Electronics,Autumn,98.700885
5,Electronics,Spring,98.619584
6,Electronics,Summer,98.902673
7,Electronics,Winter,98.352591
8,Furniture,Autumn,103.961582
9,Furniture,Spring,103.848637


In [38]:
#Calculate avg demand by category according to weather
query = '''
SELECT
    p.category,
    mc.weather_condition,
    AVG(i.demand_forecast) AS avg_demand
FROM
    InventoryFacts i
JOIN
    Products p ON i.product_id = p.product_id
    JOIN MarketConditions mc ON i.date = mc.date AND i.store_id = mc.store_id AND i.region = mc.region
GROUP BY
    p.category, mc.weather_condition
ORDER BY
    p.category, mc.weather_condition;
'''
#Print all rows
df_weather_demand = pd.read_sql_query(query, conn)
df_weather_demand.head(25)

,category,weather_condition,avg_demand
0,Clothing,Cloudy,117.222394
1,Clothing,Rainy,117.774935
2,Clothing,Snowy,118.640967
3,Clothing,Sunny,116.850354
4,Electronics,Cloudy,99.156965
5,Electronics,Rainy,95.904703
6,Electronics,Snowy,100.003790
7,Electronics,Sunny,99.508787
8,Furniture,Cloudy,103.701789
9,Furniture,Rainy,103.947000


In [39]:
#Calculate average forecasted demand per category according to region
query = '''
SELECT
    p.category, mc.region, AVG(i.demand_forecast) AS avg_demand
FROM
    InventoryFacts i
JOIN
    Products p ON i.product_id = p.product_id
    JOIN MarketConditions mc ON i.date = mc.date AND i.store_id = mc.store_id AND i.region = mc.region
GROUP BY
    p.category, mc.region
ORDER BY
    p.category, mc.region;
'''
#Print all rows
df_region_demand = pd.read_sql_query(query, conn)
df_region_demand.head(25)

,category,region,avg_demand
0,Clothing,East,118.116627
1,Clothing,North,117.712144
2,Clothing,South,117.667488
3,Clothing,West,117.030100
4,Electronics,East,98.475092
5,Electronics,North,97.972591
6,Electronics,South,98.906523
7,Electronics,West,99.158211
8,Furniture,East,103.460443
9,Furniture,North,104.116557


In [40]:
#Calculate the profit percentage earned by category
query = '''
SELECT
    p.category,
    ROUND(SUM(i.units_sold * (i.price - i.price * (i.discount / 100.0))), 2) AS profit,
    ROUND(SUM(i.units_sold * i.price), 2) AS revenue,
    ROUND(
        SUM(i.units_sold * (i.price - i.price * (i.discount / 100.0))) * 100.0 /
        NULLIF(SUM(i.units_sold * i.price), 0), 2
    ) AS profit_percentage
FROM
    InventoryFacts i
JOIN
    Products p ON i.product_id = p.product_id
GROUP BY
    p.category
ORDER BY
    profit_percentage DESC;

'''
#Print all rows
df_profit = pd.read_sql_query(query, conn)
df_profit.head(25)

,category,profit,revenue,profit_percentage
0,Electronics,1.311314e+08,1.417218e+08,92.53
1,Furniture,1.039822e+08,1.124401e+08,92.48
2,Toys,5.173887e+07,5.595318e+07,92.47
3,Clothing,2.152612e+08,2.328345e+08,92.45
4,Groceries,3.732857e+07,4.041894e+07,92.35


In [43]:
#Calculate inventory turnover ratio=sales value/Inventory level (price-discount) by category
query = '''
SELECT
    p.category,
    ROUND(SUM(i.units_sold * (i.price - i.price * (i.discount / 100.0))), 2) AS sales_value,
    ROUND(SUM(i.inventory_level), 2) AS inventory_level,
    ROUND(
        SUM(i.units_sold * (i.price - i.price * (i.discount / 100.0))) / NULLIF(SUM(i.inventory_level), 0),
        2
    ) AS inventory_turnover_ratio
FROM
    InventoryFacts i
JOIN
    Products p ON i.product_id = p.product_id
GROUP BY
    p.category
ORDER BY
    inventory_turnover_ratio DESC;
'''

df_inventory_turnover = pd.read_sql_query(query, conn)
display(df_inventory_turnover)

,category,sales_value,inventory_level,inventory_turnover_ratio
0,Clothing,2.152612e+08,6205992.0,34.69
1,Groceries,3.732857e+07,1086559.0,34.35
2,Furniture,1.039822e+08,3103300.0,33.51
3,Toys,5.173887e+07,1552597.0,33.32
4,Electronics,1.311314e+08,4006354.0,32.73


In [50]:
#Categories with highest number of units sold according to region, print the max category for each region
query = '''
SELECT
    mc.region,
    p.category,
    MAX(i.units_sold) AS max_units_sold
FROM
    InventoryFacts i
JOIN
    Products p ON i.product_id = p.product_id
    JOIN MarketConditions mc ON i.date = mc.date AND i.store_id = mc.store_id AND i.region = mc.region
GROUP BY
    mc.region
ORDER BY
    mc.region;
    '''

df_max_units_sold_by_region = pd.read_sql_query(query, conn)
display(df_max_units_sold_by_region)

,region,category,max_units_sold
0,East,Clothing,323
1,North,Clothing,342
2,South,Clothing,333
3,West,Clothing,310


In [109]:
#Add separate columns of reorder points and safety stocks for each item
#Average daily demand= inventory level+units ordered-units sold for each product
#Reorder point=average daily demand+Safety stock
#Safety stock=Maximum daily sales by category-demand forecast
query = '''
WITH category_stats AS (
    SELECT
        p.category,
        MAX(i.units_sold) AS max_daily_sales
    FROM InventoryFacts i
    JOIN Products p ON i.product_id = p.product_id
    GROUP BY p.category
),
daily_facts AS (
    SELECT
        i.date,
        i.store_id,
        i.region,
        i.product_id,
        p.category,
        i.inventory_level,
        i.units_sold,
        i.units_ordered,
        i.demand_forecast
    FROM InventoryFacts i
    JOIN Products p ON i.product_id = p.product_id
)

SELECT
    df.*,
    cs.max_daily_sales,
    -- Row-wise average daily demand based on inventory logic
    (df.inventory_level + df.units_ordered - df.units_sold) AS avg_daily_demand,
    -- Safety stock
    ROUND(cs.max_daily_sales - df.demand_forecast, 2) AS safety_stock,
    -- Reorder point = avg_daily_demand + safety_stock
    ROUND(
        (df.inventory_level + df.units_ordered - df.units_sold) + (cs.max_daily_sales - df.demand_forecast),
        2
    ) AS reorder_point
FROM daily_facts df
JOIN category_stats cs ON df.category = cs.category
ORDER BY df.date, df.store_id, df.product_id
LIMIT 1000000;
'''
#Export the above results to CSV
df_reorder = pd.read_sql_query(query, conn)
df_reorder.to_csv('reorder_points.csv', index=False)
# Fetch from SQLite
df_reorder = pd.read_sql_query(query, conn)

# Display sample
display(df_reorder.head(25))

# Products at risk of stockout
df_stockout_risk = df_reorder[df_reorder['units_ordered'] > df_reorder['reorder_point']]
display(df_stockout_risk.head(100000))



,date,store_id,region,product_id,category,inventory_level,units_sold,units_ordered,demand_forecast,max_daily_sales,avg_daily_demand,safety_stock,reorder_point
0,2022-01-01,S001,East,P0016,Clothing,189,127,125,150.47,342,187,191.53,378.53
1,2022-01-01,S001,South,P0017,Toys,70,64,68,92.68,236,74,143.32,217.32
2,2022-01-01,S001,West,P0031,Electronics,75,48,39,68.62,263,66,194.38,260.38
3,2022-01-01,S001,South,P0046,Clothing,139,117,110,128.42,342,132,213.58,345.58
4,2022-01-01,S001,East,P0057,Clothing,142,115,112,118.46,342,139,223.54,362.54
5,2022-01-01,S001,South,P0061,Clothing,180,94,83,127.95,342,169,214.05,383.05
6,2022-01-01,S001,North,P0066,Clothing,190,138,116,149.00,342,168,193.00,361.00
7,2022-01-01,S001,North,P0067,Furniture,101,93,92,120.27,251,100,130.73,230.73
8,2022-01-01,S001,South,P0068,Electronics,70,44,48,53.74,263,74,209.26,283.26
9,2022-01-01,S001,North,P0069,Clothing,58,53,42,50.31,342,47,291.69,338.69


,date,store_id,region,product_id,category,inventory_level,units_sold,units_ordered,demand_forecast,max_daily_sales,avg_daily_demand,safety_stock,reorder_point
12645,2022-03-26,S002,North,P0096,Toys,187,219,178,205.71,236,146,30.29,176.29
23047,2022-06-03,S004,West,P0067,Furniture,200,235,193,216.20,251,158,34.80,192.80
26724,2022-06-28,S001,East,P0166,Groceries,252,280,236,272.28,280,208,7.72,215.72
31306,2022-07-28,S004,South,P0116,Furniture,189,228,186,217.80,251,147,33.20,180.20
76004,2023-05-22,S004,South,P0094,Groceries,214,250,210,254.16,280,174,25.84,199.84
85136,2023-07-22,S003,North,P0175,Electronics,190,195,161,267.32,263,156,-4.32,151.68
101069,2023-11-05,S004,East,P0187,Clothing,283,309,280,329.83,342,254,12.17,266.17
108020,2023-12-22,S001,East,P0133,Clothing,265,306,251,304.37,342,210,37.63,247.63


In [101]:
query = '''
SELECT
    p.category,
    COUNT(*) AS overpriced_count
FROM InventoryFacts i
JOIN Products p
    ON i.product_id = p.product_id
JOIN MarketConditions mc
    ON i.date = mc.date AND i.store_id = mc.store_id AND i.region = mc.region
WHERE
    (i.price - (i.discount / 100.0 * i.price)) > mc.competitor_pricing
GROUP BY
    p.category
ORDER BY
    overpriced_count DESC;
'''

df_overpriced_categories = pd.read_sql_query(query, conn)
display(df_overpriced_categories)


#Return the categories with the minimum and maximum count
query = '''
SELECT
    category,
    COUNT(*) AS item_count
FROM
    InventoryFacts i
JOIN
    Products p ON i.product_id = p.product_id
GROUP BY
    category
ORDER BY
    item_count DESC;
'''

df_category_item_count = pd.read_sql_query(query, conn)
display(df_category_item_count)

,category,overpriced_count
0,Clothing,17424
1,Electronics,11911
2,Furniture,9484
3,Toys,3966
4,Groceries,3318


,category,item_count
0,Clothing,40150
1,Electronics,29200
2,Furniture,21900
3,Toys,10950
4,Groceries,7300


In [104]:
#Compare total units sold with holiday/promotion by categories
query = '''
SELECT
    p.category,
    d.is_holiday_or_promotion,
    SUM(i.units_sold) AS total_units_sold
FROM
    InventoryFacts i
JOIN
    Products p ON i.product_id = p.product_id
    JOIN Dates d ON i.date = d.date
GROUP BY
    p.category, d.is_holiday_or_promotion
    ORDER BY
        p.category, d.is_holiday_or_promotion;
'''

df_holiday_promotion_units_sold = pd.read_sql_query(query, conn)
display(df_holiday_promotion_units_sold)

,category,is_holiday_or_promotion,total_units_sold
0,Clothing,0,3590285
1,Clothing,1,640595
2,Electronics,0,2182966
3,Electronics,1,394077
4,Furniture,0,1727584
5,Furniture,1,307767
6,Groceries,0,617657
7,Groceries,1,111893
8,Toys,0,859487
9,Toys,1,159006


In [105]:
#Print category items according to region
query = '''
SELECT
    mc.region,
    p.category,
    COUNT(*) AS item_count
FROM
    InventoryFacts i
JOIN
    Products p ON i.product_id = p.product_id
    JOIN MarketConditions mc ON i.date = mc.date AND i.store_id = mc.store_id AND i.region = mc.region
GROUP BY
    mc.region, p.category
ORDER BY
    mc.region, item_count DESC;
'''

df_category_items_by_region = pd.read_sql_query(query, conn)
display(df_category_items_by_region)

,region,category,item_count
0,East,Clothing,10084
1,East,Electronics,7426
2,East,Furniture,5505
3,East,Toys,2722
4,East,Groceries,1822
5,North,Clothing,9996
6,North,Electronics,7367
7,North,Furniture,5414
8,North,Toys,2716
9,North,Groceries,1800


In [108]:
#Calculate total forecasted demand by category according to season
query = '''
SELECT
    p.category,
    d.seasonality,
    SUM(i.demand_forecast) AS total_demand
FROM
    InventoryFacts i
JOIN
    Products p ON i.product_id = p.product_id
    JOIN Dates d ON i.date = d.date
GROUP BY
    p.category, d.seasonality
ORDER BY
    p.category, d.seasonality;
    '''

df_season_demand_forecast = pd.read_sql_query(query, conn)
display(df_season_demand_forecast)





,category,seasonality,total_demand
0,Clothing,Autumn,1050854.78
1,Clothing,Spring,700899.58
2,Clothing,Summer,1053374.39
3,Clothing,Winter,1917756.86
4,Electronics,Autumn,726438.51
5,Electronics,Spring,481263.57
6,Electronics,Summer,727923.67
7,Electronics,Winter,944184.87
8,Furniture,Autumn,573867.93
9,Furniture,Spring,380086.01
